In [1]:
import os
from pathlib import Path
import shutil

from filenameVerif import *



In [2]:

import ast 

import numpy as np
import pandas as pd



from operator import sub,add
from collections import Counter, defaultdict

In [3]:
class BatchLoaderFiles():
    def __init__(self,pathFiles,batch_len =  5 ):
        self.path_files = pathFiles
        self.treated_files = []
        self.not_treatedFiles = []
        self.batch_len = batch_len
        
    def left_setDiff(lst_1,lst2):
        return  list(set(lst_1) - set(lst2))
        
    def updateNotTreatedFiles(self):
        lst_json = self.getCurFiles()
        self.not_treatedFiles = BatchLoaderFiles.left_setDiff(lst_json,self.treated_files)
        
    def getCurFiles (self ):
        lst_json = []
        if( os.path.exists(self.path_files )) :
            lst_json = [pos_json for pos_json in os.listdir(self.path_files) if pos_json.endswith('.json') and filename_isvalid(pos_json)]
        return lst_json
    
    def updateTreatedFiles(self,lst_files_treated):
        for f in lst_files_treated :
            if( f in self.not_treatedFiles ):
                self.treated_files.append(f)
                self.not_treatedFiles.remove(f)
                
    def getNextBatch(self):
        notTreated_len = len(self.not_treatedFiles) 
        if(notTreated_len <self.batch_len) : 
            self.updateNotTreatedFiles()
        if( len(self.not_treatedFiles ) <self.batch_len ): 
            return self.not_treatedFiles
        return self.not_treatedFiles[:self.batch_len]
    
                
    
                
                
    
                
    
class Test():
    def __init__ (self,_name_test , _expected , _result , f_eq = None ):
        self.test_name = _name_test
        self.expected = _expected 
        self.result = _result
        self.f_eq = f_eq
    def message_testFailed(self):
        return "TEST " + self.test_name +" FAILED " 
    def message_diffExpectedResult(self):
        return "Return " + str(self.result) + " vs " + "Expected " + str(self.expected )
    
    def message_testSuccess(self):
        return "TEST " + self.test_name +" PASSED "
    
    
    def print_resultTest( self   ):
            if(self.result!=self.expected if self.f_eq is None  else not(self.f_eq ( self.result,self.expected))):
                print(self.message_testFailed())
                print(self.message_diffExpectedResult())
            else :
                print(self.message_testSuccess())
    
    

class Mock_BatchLoader():
    
    def delete_content_files(str_path):
        path_files = Path(str_path)
        path_files = Test_BatchLoader.rm_ifExist(path_files)
        
    def mock_create_files(str_path ,lst_files):
        path_files = Path(str_path)
        if not(path_files.exists() and path_files.is_dir()):
            os.mkdir(str_path)
        for str_file in lst_files:
            with open(os.path.join(str_path,str_file), 'w') as f:
                f.write("{}")
    
class Test_BatchLoader(Test):
    def __init__(self):
        name_test , result,expected = Test_BatchLoader.test_leftSetDiff()
        test_setDiff = Test(name_test,result,expected)
        test_setDiff.print_resultTest()
        
        name_test , result,expected = Test_BatchLoader.test_getCurFiles()
        test_setDiff = Test(name_test,result,expected,(lambda x,y : sorted(x)==sorted(y)))
        test_setDiff.print_resultTest()
        
        name_test , result,expected = Test_BatchLoader.test_updateTreatedFiles()
        test_setDiff = Test(name_test,result,expected,(lambda x,y : sorted(x)==sorted(y)))
        test_setDiff.print_resultTest()
        
        
        name_test , result,expected = Test_BatchLoader.test_updateNotTreatedFiles()
        test_setDiff = Test(name_test,result,expected,(lambda x,y : sorted(x)==sorted(y)))
        test_setDiff.print_resultTest()
        
        name_test , result,expected =Test_BatchLoader.test_getNextBatch()
        test_setDiff = Test(name_test,result,expected)
        test_setDiff.print_resultTest()
        
        
        
        
    def test_leftSetDiff():
        lst_1 = ["2_113_12-6_1681562526.json","1_114_12-6_1681562526.json","1_111_12-6_1681562526.json","6_110_12-6_1681562526.json"]
        lst_2 = ["2_113_12-6_1681562526.json","9_114_12-6_1681562526.json","1_111_12-6_1681562526.json","6_110_12-6_1681562526.json"]
        return "leftSetDiff" , BatchLoaderFiles.left_setDiff(lst_1,lst_2),["1_114_12-6_1681562526.json"]
    
    
    def rm_ifExist(str_path):

        path_files = Path(str_path)
        if path_files.exists() and path_files.is_dir():
            shutil.rmtree(path_files)
        return path_files
    
            
    def test_getCurFiles():
        str_path =os.path.join(os.getcwd(),"tmp")
        lst_files = ["2_113_12-6_1681562526.json","9_114_12-6_1681562526.json","1_111_12-6_1681562526.json","6_110_12-6_1681562526.json"]
        path_files = Path(str_path)
        Mock_BatchLoader.delete_content_files( str_path)
        Mock_BatchLoader.mock_create_files(str_path,lst_files)
        batch_loader = BatchLoaderFiles( os.path.join(os.getcwd(),str_path))
        return "getCurFiles" , batch_loader.getCurFiles (),lst_files
    
    def test_updateTreatedFiles():
        str_path =os.path.join(os.getcwd(),"tmp")
        lst_files_1 = ["2_113_12-6_1681562526.json","9_114_12-6_1681562526.json","1_111_12-6_1681562526.json","6_110_12-6_1681562526.json"]
       
        Mock_BatchLoader.delete_content_files(str_path)
        Mock_BatchLoader.mock_create_files(str_path,lst_files_1)
        
        btlf= BatchLoaderFiles(str_path)
        btlf.updateNotTreatedFiles()
        
        btlf.updateTreatedFiles(lst_files_1[:-2])
        
        return "updateTreatedFiles",btlf.treated_files,lst_files_1[:-2]
    
    def test_updateNotTreatedFiles():
        
        str_path =os.path.join(os.getcwd(),"tmp")
        lst_files_1 = ["2_113_12-6_1681562526.json","9_114_12-6_1681562526.json","1_111_12-6_1681562526.json","6_110_12-6_1681562526.json"]
       
        Mock_BatchLoader.delete_content_files(str_path)
        Mock_BatchLoader.mock_create_files(str_path,lst_files_1)
        
        btlf= BatchLoaderFiles(str_path)
        btlf.updateNotTreatedFiles()
        
        
        btlf.updateTreatedFiles(lst_files_1[:-2])
        
        
        
        lst_files_2 = ["2_403_12-6_1681562526.json","9_140_12-6_1681562526.json","1_001_12-6_1681562526.json","6_119_12-6_1681562526.json"]
       
        Mock_BatchLoader.mock_create_files(str_path,lst_files_2)
        btlf.updateNotTreatedFiles()
        
        return "updateNotTreatedFiles",btlf.not_treatedFiles,lst_files_2 + lst_files_1[-2:]
        
        
    def test_getNextBatch():
        str_path =os.path.join(os.getcwd(),"tmp")
        lst_files_1 = ["2_113_12-6_1681562526.json","9_114_12-6_1681562526.json","1_111_12-6_1681562526.json","6_110_12-6_1681562526.json"]
       
        Mock_BatchLoader.delete_content_files(str_path)
        Mock_BatchLoader.mock_create_files(str_path,lst_files_1)
        
        btlf= BatchLoaderFiles(str_path,batch_len =2)
        batch_1 = btlf.getNextBatch()
        btlf.updateTreatedFiles(btlf.getNextBatch())
        
        
        batch_2 = btlf.getNextBatch()
        btlf.updateTreatedFiles(btlf.getNextBatch())
        
        
        return "getNextBatch",sorted(batch_2)!=sorted(batch_1) ,True
        
        
    
    
        
        
        
    

In [4]:
test_batch =Test_BatchLoader()

TEST leftSetDiff PASSED 
TEST getCurFiles PASSED 
TEST updateTreatedFiles PASSED 
TEST updateNotTreatedFiles PASSED 
TEST getNextBatch PASSED 


In [5]:
import pandas as pd 
import json
from myutil  import *
import os 
import numpy as np


'''
ReadActions :
    - STATIC :
        - MAP_SIZE : nombre de cases sur la map
        - intervals_ends : fin des intervalles des types d'actions ( e.g Times : 100 ,  Buffs: 200 , ... )
        - intervals_dict : dictionnaire inversé de intervals_dict ( e.g { {START_FIGHT: 0 ,...,USE_WEAPON: 16  , LOST_PT : 100 , ... , NOVA_VITALITY :  112 , ... }  }# le fichier filename_dfActions loader 
        - intervals_dictRev : dictionnaire des types d'actions ( e.g {0:START_FIGHT ,...,16: USE_WEAPON , ... } # le fichier filename_dfActions loader reversed
        - filename_dfActions : nom du fichier json contenant les actions de chaque types d'actions ainsi que la taille des intervalles 
        - ex_idx : liste des index des membres du json  a exclure (e.g [0,2,3]<=> interval_size , Funs , Others )
        - lst_col : liste des colonnes non calculer (directement accessible via l'outcome ) à stocker dans le dataframe
        - id_col : liste des colonnes contenant les ids identifiants les lignes du dataframe
        - deg_col : liste des colonnes contenant les degats infligés ou subis par les entités (parmis les colonnes décritent dans lst_col)
        - audit_col : liste des types d'actions performer pendant le tour (e.g [0,0,1,2,1,1] <=> [df.cols[0] , df.cols[0], df.cols[1] , ...] <=> [Times , Times , Buffs , ...])

    - NON STATIC :
        - numGen : numéro de la génération
        - numFight : numéro du combat
        - beg_notCodeAct_col : index indiquant le début des colonnes qui ne sont pas des liste de codes d'actions
        - entities_ids : les ids réelle des entités (telle que spécifié dans le scénario["entities"][i]["id"] ) impliquées dans le fight du dataframe 
        - int_col_idx_in_lst_col : liste des index des colonnes contenant des ints dans lst_col 

'''
class ReadActions :
    MAP_SIZE = 612 
    intervals_ends = dict()
    intervals_dict = dict()
    intervals_dictRev = dict()  
    filename_dfActions = os.path.join("..","data","df_Action.json")
    filename_dfValueActions = os.path.join("..","data","df_value_Action.json")
    
    dict_dfValue = json.load(open( filename_dfValueActions))
    ex_idx = [0,2,3]
    lst_col = ['LOST_LIFE','LIFE','MOVE_TO','WEAPON_DEG','CHIP_DEG','MAPS','WEAPON_ID','CHIP_ID']
    _cols = ['MOVE_TO','MAPS','WEAPON_ID','CHIP_ID']
    id_col = ['ID_GEN','ID_FIGHT','ID_ENTITY','ID_TURN']
    deg_col = ['LOST_LIFE','LIFE','WEAPON_DEG','CHIP_DEG']
    audit_col = ["ACTIONS"]
    
    #maps = [ [0]*MAP_SIZE ,[0]*MAP_SIZE]
    
    def __init__(self,ids :list ,numGen :int , numFight:int ,path_atm :str ):
        self.numGen = numGen
        self. numFight = numFight
        self.numTurn = 0
        ReadActions.init_intervals(ReadActions.filename_dfActions)#,ReadActions.ex_idx)
        self.nb_type = len(ReadActions.intervals_ends)
        self.lst_col_code = [list(ReadActions.intervals_ends.keys())[i] for i in range(1,len(ReadActions.intervals_ends)) ] 
        self.df = pd.DataFrame(columns=self.lst_col_code+ReadActions.lst_col+ReadActions.id_col+ReadActions.audit_col)
        self.beg_notCodeAct_col = len(self.df.columns) - len(ReadActions.lst_col)-len(ReadActions.id_col)-1
        self.entities_ids = ids 
        self.setIntCol()
        
    def setDF(self,df2):
        if(df2.columns == self.df.columns):
            self.df = pd.concat([self.df , df2])
            
        
    def setIntCol(self):
        self.int_col_idx_in_lst_col = []
        for i,d in enumerate(self.df.columns.tolist()) :
            if d in ReadActions.deg_col :
                self.int_col_idx_in_lst_col+=[i]
        self.int_col_idx_in_lst_col.sort()


    @property
    def getNumGen(self):
        return self.numGen
    
    @property
    def getNumFight(self):
        return self.numFight
    
    @property
    def getNumTurn(self):
        return self.numTurn
    
    @property
    def getDF(self):
        return self.df
    
    @property
    def getEntitiesIds(self):
        return self.entities_ids
    



    @staticmethod
    def emptyIntervalsEnd():
            ReadActions.intervals_ends.update({'BEGIN':0})


    
    @staticmethod
    def init_intervals(filename_json_dfA :str ,ex_idx=[]):
            """
            filename_json_dfA : fichier json contenant les actions de chaque types d'actions ainsi que la taille des intervalles
            ex_idx : liste des index des membres du json  a exclure (e.g [0,2,3]<=> interval_size , Funs , Others )

            Description : initialise les attributs intervals_dict , intervals_dictRev , intervals_ends de la classe ReadActions grâce au fichier json passer en paramètre
            """
            if( isJsonFile( filename_json_dfA ) ):
                ReadActions.emptyIntervalsEnd()
                #Load json file
                with open(filename_json_dfA, 'r') as f:
                    df_actions_json = json.load(f)
                interval_size = df_actions_json["interval_size"]

                #ignore interval_size from dictionnary
                del df_actions_json["interval_size"]

                #ignore ex_idx from dictionnary
                for to_ignore in ex_idx :
                    del df_actions_json[to_ignore]

                #extract info from dict and load it in class attributes 

                idx_keyOf_action = 0
                idx_intcodeOf_action = 1
                for i,group_of_actions in enumerate(df_actions_json.items())  :
                    id_group,lst_actions=group_of_actions
                    
                    for action in lst_actions.items():
                        ReadActions.intervals_dict.update({action[idx_keyOf_action]:action[idx_intcodeOf_action]})

                    ReadActions.intervals_ends.update({id_group:(interval_size+1)*(i+1)})
                ReadActions.intervals_dictRev = reversedict(ReadActions.intervals_dict)
            else :
                errorMessagePyth("file"+filename_json_dfA+" must be a json file")
        
     
    
    @staticmethod
    def whileToken(str_token : str , lst_actions :list ):
        """
        str_token : le string correspondant au token a chercher dans lst_actions ( un token est exprimer sous la forme d'un int dans lst_actions)
        lst_actions : liste d'actions  a parcourir   ( lst_action[i][0] == code_token ) 

        Description : parcours lst_actions jusqu'a ce que le token soit trouvé ou que la fin de la liste soit atteinte 
        Return : l'index de l'action token dans lst_actions tel que ( lst_action[index][0] == code_token )  ou -1 si le token n'a pas été trouvé
        """
        for i in range(len(lst_actions)) :
            action = lst_actions[i]
            if(action[0] == ReadActions.intervals_dict.get(str_token)):
                i+=1
                return i
        return notFound()
    

    @staticmethod        
    def isEndFight(act_token: int):
        return act_token == ReadActions.intervals_dict.get("END_FIGHT")
    
    @staticmethod 
    def isBeginFight(act_token: int):
        return act_token == ReadActions.intervals_dict.get("START_FIGHT")
    
    @staticmethod 
    def isBeginTurn(act_token: int):
        return act_token == ReadActions.intervals_dict.get("NEW_TURN")
    
    @staticmethod 
    def isEndTurn(act_token: int):
        return ReadActions.isEndFight(act_token) or ReadActions.isBeginTurn(act_token ) or act_token == ReadActions.intervals_dict.get("END_TURN")
    

    @staticmethod
    def getDfLstTypeActions():
        return list([[]])
    
    @staticmethod
    def getDfLstAudit():
        return list([[]])
    
    @staticmethod
    def getDfLstInLstCol():
        return [0]
    
    @staticmethod
    def getDfLstNotInLstCol():
        return []
    

    def getNewRow(self):
        lst =[]

        #-----------------Type Actions [Times,Buffs , ... ] -----------------
        idx = 0 
        idx_end = self.beg_notCodeAct_col 
        #Type of actions , e.g Times,Buffs,Funs,Effects , Others 
        for i in range (idx,idx_end):
            lst.append(ReadActions.getDfLstTypeActions())

        t=0
        #----------------- Lst Col -----------------
        idx = idx_end
        idx_end +=len(ReadActions.lst_col)
        for i in range (idx , idx_end):
            if i in self.int_col_idx_in_lst_col[t:] :
                lst.append(ReadActions.getDfLstInLstCol())
                t+=1
            #elif ReadActions.lst_col[i-idx] in ["WEAPON_ID","CHIP_ID"]:
            #    lst.append(notSetId())
            else :
                lst.append(ReadActions.getDfLstNotInLstCol())

        #----------------- Not in Lst Col -----------------
        idx = idx_end
        idx_end += len(ReadActions.id_col)
        for i in range (idx ,  idx_end):
            lst.append(notSetId())

        #----------------- Audit -----------------
        idx = idx_end
        idx_end += len(ReadActions.audit_col)
        for i in range (idx,idx_end):
            lst.append(ReadActions.getDfLstAudit())#actions



        return lst
    
    def idxNextRow(self):
        return self.getCurrentIDX()+1
    
    def addNewRow(self):
        self.df.loc[self.idxNextRow()]=self.getNewRow()
    
        
    def addNewTurn(self,idx_turn: int):
        """
        idx_turn : index du tour a ajouter
        Description : ajoute une nouvelle ligne et set sont indexe de tour à idx_turn
        """
        self.addNewRow()
        #self.numTurn = idx_turn
        self.setColumn ('ID_TURN', idx_turn )
        
    def getCurrentIDX (self):
        return len(self.df)-1

    def setColumn ( self , str_col:str ,val):
        """
        Description : set la colonne str_col de la ligne courante à val
        """
        if ( strColInCols(str_col ,self.df)):
            self.df.loc[self.getCurrentIDX(),str_col]= val 
    

    def addToColumn(self,str_col:str,val,pos:int=0):
        """
        val : valeur a ajouter a la liste de la colonne str_col de la ligne courante
        pos : position de la valeur a ajouter dans la liste de la colonne str_col de la ligne courante (par defaut 0)

        Description : ajoute une valeur a la liste de la colonne str_col de la ligne courante
        """
        if (strColInCols(str_col ,self.df)):
            lst=self.df.loc[self.getCurrentIDX(),str_col]

            if( len(lst )< pos):
                errorMessagePyth(self,"addToColumn" , "index out of range")

            if( not(bool(lst)) or len(lst )== pos):
                lst+=[0]
            
            lst[pos]+= val  

        return pos 
            
            
    def addToColumnList(self,str_col:str,val,pos:int=0):
        """
        Attention : on embedde la valeur dans une liste car la colonne str_col est de type liste 
        """
        if(len(self.df[str_col]) == 0  or not(isinstance(self.df[str_col][0],list))):
            errorMessagePyth(self,"addToColumnList" , "column "+str_col+" must be a list")
            return badInsert()
        else :
            return self.addToColumn(str_col , [val],pos) 

    def extendColum(self,str_col:str,val:list):
        """
        Description : on ajoute une liste a la liste de la colonne str_col de la ligne courante
        """
        if (strColInCols(str_col ,self.df)):
            self.df.loc[self.getCurrentIDX(),str_col].extend(val)
       

    def addToAudit(self , idx:int , idx_of_actionType :int ,   act_token:int  ):
        """
        Description : on ajoute l'index du type de l'action courante dans la colonne audit correspondante
        """
        err_code=self.addToColumnList (  ReadActions.audit_col[idx],idx_of_actionType)
        if(err_code == badInsert()):
            errorMessagePyth(self, "addToCategory" , "bad insert")
            
    def addToGroupAction(self,idx_of_actionType:int , act_token:int):
        err_code = self.addToColumnList (list(ReadActions.intervals_ends.keys())[idx_of_actionType],act_token)
        if(err_code == badInsert()):
            errorMessagePyth(self, "addToCategory" , "bad insert")




    def setIDs(self,entity_id:int):
        """
        Description : set les colonnes IDs de la ligne courante sauf ID_TURN 
        """
        self.setColumn ('ID_GEN',self.numGen)
        self.setColumn ('ID_FIGHT',self. numFight)
        self.setColumn ('ID_ENTITY',self.getEntityID(entity_id))
        #self.setColumn ('ID_TURN',self.idx_turn)
        
    def getEntityID(self ,entity_id:int ):
        """
        Description : retourne l'ID de l'entité entity_id si elle existe sinon retourne outOfRange()
        """
        if len(self.entities_ids) > entity_id :
            return self.entities_ids[entity_id]
        
        else :
            errorMessagePyth(self,"getEntityID" , "index out of range")
            return outOfRange()
        
    @staticmethod
    def getIdxCodeToken():
        return 0
    @staticmethod
    def getIdxEntityToken():
        return 1
    
    @staticmethod
    def getCodeToken(_lst :list):
        return _lst[ReadActions.getIdxCodeToken()]
    @staticmethod
    def getEntityToken(_lst :list):
        return _lst[ReadActions.getIdxEntityToken()]
    
    def getTurn( self,lst_actions :list ,idx_turn :int ):
        entity_id = -1 
        
        # continue jusqu'au token LEEK_TURN
        nbActionRead=ReadActions.whileToken("LEEK_TURN",lst_actions)

        k = nbActionRead

        if(k==notFound()):
            return k , None 
        
        act_token = ReadActions.getCodeToken(lst_actions[k])
        entity_id =  ReadActions.getEntityToken(lst_actions[k-1])
        
        self.addNewTurn(idx_turn)
        self.setIDs(entity_id)

        #si le tour est vide : 
        if(self.isEndTurn(act_token)):
            return k , entity_id 
        
        idx=0
        while k  < len(lst_actions):

            act_token = ReadActions.getCodeToken(lst_actions[k])

            #si le tour est fini : 
            if(self.isEndTurn(act_token) ):
                return k , entity_id
            

            for i in range(1,len(ReadActions.intervals_ends)):
                #idx_group_actions=i
                interval = [list(ReadActions.intervals_ends.values())[i-1],list(ReadActions.intervals_ends.values())[i]]
                if(act_token  in ReadActions.intervals_dictRev and isInInterval(act_token , interval )) :
                    
                    _id = ReadActions.getEntityToken(lst_actions[k])
                    self.addToAudit(ReadActions.audit_col.index("ACTIONS"),i,act_token)
                    self.addToGroupAction(i , act_token)

        

                    if(list(ReadActions.intervals_ends.keys())[i] == "Times"):

                        if(ReadActions.intervals_dictRev.get(act_token) == "MOVE_TO"):
                            lst_mv = lst_actions[k][ReadActions.dict_dfValue[str(act_token)]["name_fields"].index("path")]
                            self.extendColum('MOVE_TO',[lst_mv[0],len(lst_mv),lst_mv[-1]])
                            self.extendColum('MAPS',lst_mv)
                            
  
                            
                    elif(list(ReadActions.intervals_ends.keys())[i] == "Buffs"):
                        if(ReadActions.intervals_dictRev.get(act_token) == 'HEAL'):
                            self.addToColumn('LIFE',lst_actions[k][ReadActions.dict_dfValue[str(act_token)]["name_fields"].index("life")])
                        elif(ReadActions.intervals_dictRev.get(act_token) in ['LOST_LIFE','NOVA_DAMAGE','DAMAGE_RETURN','LIFE_DAMAGE','POISON_DAMAGE',' AFTEREFFECT']):
                            self.addToColumn('LOST_LIFE',lst_actions[k][ReadActions.dict_dfValue[str(act_token)]["name_fields"].index("pv")])
                    elif(list(ReadActions.intervals_ends.keys())[i] == "Effects"):
                        if(ReadActions.intervals_dictRev.get(act_token) == 'ADD_WEAPON_EFFECT'):
                            self.addToColumn('WEAPON_DEG',lst_actions[k][ReadActions.dict_dfValue[str(act_token)]["name_fields"].index("value")])
                            self.extendColum('WEAPON_ID',[_id])
                        elif(ReadActions.intervals_dictRev.get(act_token) == 'ADD_CHIP_EFFECT'):
                                self.addToColumn('CHIP_DEG',lst_actions[k][ReadActions.dict_dfValue[str(act_token)]["name_fields"].index("value")])
                                self.extendColum('CHIP_ID',[_id])
                                
                    
                    
            k+=1
            idx+=1
        return k , entity_id

                            
                    
    def getFight ( self,lst_actions:list ):
        nbReads=ReadActions.whileToken("START_FIGHT",lst_actions)

        k=nbReads

        idx_turn = 0 
        while k < len(lst_actions):
            act_token =ReadActions.getCodeToken(lst_actions[k])
            if(self.isEndFight(act_token )):
                return k 
            nbReads , ent_id = self.getTurn(lst_actions[k:],idx_turn)
            if(nbReads==-1):
                break
            k+=(nbReads+1)
            if(k < len(lst_actions)):
                act_token =ReadActions.getCodeToken(lst_actions[k])
                if(self.isBeginTurn(act_token)):
                    idx_turn +=1


In [6]:
def parse_fileOutcome_retDF(example_1_outcome,pathOutcome,quatuor=None,ancien=False):  
        if quatuor is None and b_filename_isvalid(example_1_outcome):
            quatuor= filename_isvalid(example_1_outcome)  
        
        if(len(quatuor) == 4):
            numGen , numFight , ids_str , _example_1_outcome = quatuor
            ids = splitIds(ids_str)
            rda = ReadActions(ids ,numGen, numFight , _example_1_outcome )

            example_1_outcome_path = os.path.join(pathOutcome , example_1_outcome)

            dct_outcome = dict()
            with open(example_1_outcome_path) as _f2:
                dct_outcome = json.load(_f2)
            if(isinstance(dct_outcome,str)):
                dct_outcome =json.loads(dct_outcome )
            lst_actions =None 
            if("fight" in dct_outcome):
                dct_outcome = dct_outcome["fight"]
            if("actions" in dct_outcome):
                lst_actions = dct_outcome["actions"]
            #lst_actions = dct_outcome["actions"]if ancien else dct_outcome["fight"]["actions"]
            if(bool(lst_actions )):
                rda.getFight (lst_actions)
                return rda
            else :
                return None 


In [7]:
def lst_atmLife(df_cpy,life_entities , str_col ="ADD_LIFE" ):
    lst_res = []
    nb_elem = len(life_entities)
    for i , add_life in enumerate(df_cpy[str_col ]) :

        if i >= nb_elem :
                    lst_res.append(list(map(add,add_life,lst_res[-nb_elem])))

        else:
            lst_res.append(list(map(add,add_life,life_entities[len(lst_res)-nb_elem])))
    return  lst_res 

In [8]:

def df_addLife_atmLife(df , life_entities):
    df_cpy = df.copy()
    df_cpy["ADD_LIFE"] = df_cpy.apply(lambda row: list( map(sub, row["LIFE"] , row["LOST_LIFE"])), axis=1)
    df_cpy["ATM_LIFE"]=lst_atmLife(df_cpy , life_entities)
    return df_cpy 

In [9]:

def convert_int_list(x):
    return ast.literal_eval(x)


In [10]:
def getMaxID(str_col):
    path_data = "D:\\Master\\S2\\TER\\analyse\\data"
    if(str_col == "MAPS"):
        return [i for i in range(612)]
    if(str_col == "WEAPON_ID"):
        dct_ = json.load(open(os.path.join(path_data,"weapons"+".json")))
        return [e["item"] for e in dct_.values()] #list(map(int,list(dct_.keys())))
    if(str_col == "CHIP_ID"):
        dct_ = json.load(open(os.path.join(path_data,"chips"+".json")))
        return list(map(int,list(dct_.keys())))

In [11]:
def init_folder_genSplit(nb_folders_gen,nb_folders_turn,root_path =os.getcwd() ):
    if not(os.path.exists(root_path)) or  not(os.path.isdir(root_path)):
        os.makedirs(root_path)
    for partGen in range(nb_folders_gen):
        path_gen=os.path.join(root_path,str(partGen))
        print(path_gen)
        if not(os.path.exists(path_gen)) or  not(os.path.isdir(path_gen)):
            os.mkdir(path_gen)
        '''for partTurn in range(nb_folders_turn):
            path_turn= os.path.join(path_gen,str(partTurn))
            print(path_turn)
            if not(os.path.exists(path_turn)) or  not(os.path.isdir(path_turn)):
                os.mkdir(path_turn)'''
    return root_path

def clean_folder_genSplit(root_path):
    if os.path.exists(root_path) and  os.path.isdir(root_path):
        shutil.rmtree(root_path)

In [12]:
from collections import Counter
#dict_part={"PART_TURN":0,"PART_GEN":6}
def df_cols (path_root_f_  ,dict_part,index_col_=[i for i in range(len(ReadActions.id_col))],ignore_col = ["MOVE_TO"]):

    path_f= os.path.join(os.path.join(path_root_f_ ,str(dict_part["PART_GEN"])),str(dict_part["PART_TURN"]))
    
    dct_res=dict()
    
    for str_col_ in ReadActions._cols:

        if(not(str_col_ in ignore_col )):
            acc_counter = Counter()
            f_name_out = os.path.join(path_f ,str_col_)+".csv"

            df = pd.read_csv(f_name_out,index_col=index_col_,sep=";",encoding="utf-8",converters={str_col_: convert_int_list})#DataFrame({'data': [np.random.randn(3) for i in range(N)]})

            batch_size = 2500

            for i in range(0, len(df), batch_size):

                batch = df.loc[df.index[i:i+batch_size-1], str_col_].tolist()

                batch_size = len(batch)


                # Concaténation des listes
                concatenated_data = np.concatenate(batch)
                my_counter = Counter(concatenated_data)

                acc_counter += my_counter

            res_counter = Counter({i: 0 for i in getMaxID(str_col_)})#unecessary if skip nan after 
            res_counter.update(acc_counter)
            res_counter.update(dict_part)
            #res_counter.update({"PART_GEN":part_gen_})
            
            
            res_counter = {str(k): v for k, v in res_counter.items()}
            
            
            #path_f_csv = os.path.join(path_root_f_,str_col_+'.csv')
            '''try:
                df_write = pd.read_csv(path_f_csv ,sep=";",encoding="utf-8", index_col=[0])
            except :
                df_write = pd.DataFrame(columns=list(res_counter.keys()))
            '''

            
            # Append the new row to the DataFrame
            #df_write = df_write.append(res_counter,ignore_index=True)
            

            
            #df_dct_ = pd.DataFrame([res_counter])
            
            #df_write =pd.concat([df_write, df_dct_],ignore_index=True)
            
            
            #display(df_write)
            

            # Write the DataFrame back to the CSV file
            #df_write.to_csv(path_f_csv ,sep=";",encoding="utf-8")
            dct_res.update({str_col_ : res_counter})
    return dct_res
        

In [13]:
def df_deg_col (path_root_f_  ,dict_part,index_col_=[i for i in range(len(ReadActions.id_col))]):

    try:
    
        path_f= os.path.join(os.path.join(path_root_f_,str(dict_part["PART_GEN"])),str(dict_part["PART_TURN"]))
        
        dct_res = dict()
        
        for str_col_ in ReadActions.deg_col:

            f_name_out = os.path.join(path_f ,str_col_)+".csv"

            df = pd.read_csv(f_name_out,index_col=index_col_,sep=";",encoding="utf-8",converters={str_col_: convert_int_list})#DataFrame({'data': [np.random.randn(3) for i in range(N)]})


            # Définition de la taille des batchs
            batch_size = 2500

            # Initialisation des accumulateurs
            mean_acc = 0.0
            variance_acc = 0.0
            count = 0

            # Boucle sur les batchs
            for i in range(0, len(df), batch_size):

                batch = df.loc[df.index[i:i+batch_size-1], str_col_].tolist()

                batch_size = len(batch)


                # Concaténation des listes
                concatenated_data = np.concatenate(batch)
                batch_mean = np.mean(concatenated_data)
                batch_var = np.var(concatenated_data)
                batch_count = len(concatenated_data)


                # Mise à jour des accumulateurs
                delta = batch_mean - mean_acc
                mean_acc += delta * batch_count / (count + batch_count)
                variance_acc += batch_var * batch_count + delta**2 * count * (batch_count / (count + batch_count))
                count += batch_count

            # Calcul de la moyenne, de la variance et de l'écart-type
            mean = round(mean_acc,2)
            _tu_ = variance_acc /(count - 1) if count > 1 else 0
            variance = round(_tu_,2)
            std_dev = round(np.sqrt(variance),2)
            
            res_counter = dict({"MEAN":mean,"VAR":variance,"STD_DEV":std_dev})
            res_counter.update(dict_part)


            #path_f_csv = os.path.join(path_root_f_,str_col_+'.csv')

            '''try:
                df_write = pd.read_csv(path_f_csv ,sep=";",encoding="utf-8", index_col=[0])
            except :
                df_write = pd.DataFrame(columns=list(dict_part.keys()))
            '''
                
            # Append the new row to the DataFrame

            
            dct_res.update({str_col_ : res_counter})
            
            

            #df_write = df_write.append(res_counter,ignore_index=True)
            #df_write =pd.concat([df_write, pd.DataFrame([res_counter])],ignore_index=True)
            #df_write.to_csv(path_f_csv ,sep=";",encoding="utf-8")
    except  :
        print("UnsortedIndexError")
        
    return dct_res
    
    
#display(df_stat_mean)

In [14]:
SIZE_LIMIT = 3*pow(10,6)

In [15]:
def treate_batch(dct_num,root_path_gen_turn,batch_,dict_len_part,nbPartTurn,life_entities,str_path=os.getcwd(),ignore_col = ["MOVE_TO"],index_col_=[i for i in range(len(ReadActions.id_col))]):
    len_part_turn = dict_len_part["LEN_PART_TURN"]
    len_part_gen= dict_len_part["LEN_PART_GEN"]
    
    dict_part={"PART_GEN":-1,"PART_TURN":-1}
    lst_dct_deg_col = {key: [] for key in ReadActions.deg_col}
    lst_lst_col  = {key: [] for key in ReadActions._cols if not (key  in ignore_col )}
        
    for filename_batch in batch_:    
        if b_filename_isvalid(filename_batch):
                    _quatuor= filename_isvalid(filename_batch)  
        _rda = parse_fileOutcome_retDF(filename_batch,str_path,quatuor=_quatuor)
        if(_rda is None ):
            print(filename_batch)
            print("ERREUR PARSE ")
            continue
        df_final=df_addLife_atmLife(_rda.df,life_entities)
        dict_part["PART_GEN"]=int(_quatuor[0])//len_part_gen
        path_gen_csv = os.path.join(root_path_gen_turn,str(dict_part["PART_GEN"]))
        for iz in range(nbPartTurn):
            dict_part["PART_TURN"]=iz
            partie = df_final.loc[(df_final['ID_TURN'] > len_part_turn*iz) & (df_final['ID_TURN'] < len_part_turn*(iz+1))]
            if partie.shape[0] > 0 :
                path_turn_csv = os.path.join(path_gen_csv,str(iz))
                if not(Path(path_turn_csv).exists()):
                    os.mkdir(path_turn_csv)
                for str_col in list(set(partie.columns)- set(ReadActions.id_col)):
                    path_csv_col = os.path.join(path_turn_csv,str_col)
                    final_path_csv =path_csv_col+".csv"
                    sub_partie = partie[[str_col]+ReadActions.id_col]
                    sub_partie.set_index(ReadActions.id_col,inplace=True)

                    if Path(final_path_csv).exists():

                        concat_df = pd.read_csv(final_path_csv,sep=";", index_col=index_col_,encoding="utf-8")
                        final_df = pd.concat([concat_df,sub_partie])
                        final_df.to_csv(final_path_csv,sep=";",encoding="utf-8")
                    else:
                        sub_partie.to_csv(final_path_csv,sep=";",encoding="utf-8")


                #print(dict_part)
                
                dct_deg = df_deg_col (root_path_gen_turn,dict_part,index_col_=index_col_)
                for _k ,_v in dct_deg.items():
                    lst_dct_deg_col[_k].append(_v)
                    
                dct_lst = df_cols(root_path_gen_turn,dict_part,index_col_=index_col_)
                for _k ,_v in dct_lst.items():
                    __k= int(_k) if _k.isnumeric() else _k
                    lst_lst_col[str(__k)].append(_v)
                    
    for _k , _v in lst_dct_deg_col.items():
        path_r_csv =  os.path.join(root_path_gen_turn,str(_k))
        path_f_csv = os.path.join(path_r_csv,str(dct_num[_k])+'.csv')
        try:
            if(os.path.getsize(path_f_csv) > SIZE_LIMIT):
                df_write = pd.DataFrame(columns=list(_v[0].keys()))#columns=["MEAN","VAR","STD_DEV"]+list(dict_part.keys()))
                dct_num[_k]+=1
                path_f_csv = os.path.join(path_r_csv,str(dct_num(_k))+'.csv')
            else:
                df_write = pd.read_csv(path_f_csv ,sep=";",encoding="utf-8", index_col=[0])
        except :
            df_write = pd.DataFrame(columns=list(_v[0].keys()))#columns=["MEAN","VAR","STD_DEV"]+list(dict_part.keys()))
        df_write = df_write.append(_v, ignore_index=True)
        df_write.to_csv(path_f_csv ,sep=";",encoding="utf-8")
        
    for _k , _v in lst_lst_col.items():
        path_r_csv =  os.path.join(root_path_gen_turn,str(_k))
        path_f_csv = os.path.join(path_r_csv,str(dct_num[_k])+'.csv')
        try:
            if(os.path.getsize(path_f_csv) > SIZE_LIMIT):
                df_write = pd.DataFrame(columns=list(_v[0].keys()))#getMaxID(str(_k))+list(dict_part.keys()))
                dct_num[_k]+=1
                path_f_csv = os.path.join(path_r_csv,str(dct_num[_k])+'.csv')
            else:
                df_write = pd.read_csv(path_f_csv ,sep=";",encoding="utf-8", index_col=[0])
        except :
            df_write = pd.DataFrame(columns=list(_v[0].keys()))#getMaxID(str(_k))+list(dict_part.keys()))
        df_write = df_write.append(_v, ignore_index=True)
        df_write.to_csv(path_f_csv ,sep=";",encoding="utf-8")
        
        

In [16]:
def treat_files_byBatch(life_entities,str_path_outcome ,dict_len_part,nbPartTurn,root_path_gen_turn,len_batch = 100):
    btlf= BatchLoaderFiles(str_path_outcome,batch_len =len_batch)
    batch_ = btlf.getNextBatch()
    count_f_outcome=0
    count_batch_outcome=0
    dct_num={k:0 for k in ReadActions.lst_col }
    while(len(batch_) >= len_batch):
        count_batch_outcome += 1
        '''
        for _filename in batch_ :
            count_f_outcome+=1
            if b_filename_isvalid(_filename):
                numGen , numFight , ids_str , outcome_filename = filename_isvalid(_filename)  
                ids = splitIds(ids_str)'''

        print(batch_)
        treate_batch(dct_num,root_path_gen_turn,batch_,dict_len_part,nbPartTurn,life_entities,str_path=str_path_outcome)        
        btlf.updateTreatedFiles(batch_)
        batch_ = btlf.getNextBatch()

In [17]:
def init_resume_folder(path_root_f):
    for str_col in ReadActions.lst_col+ReadActions.audit_col :
        path_r_csv = os.path.join(path_root_f,str_col)
        os.mkdir(path_r_csv)
    

In [18]:


genetic_path = "D:\\Master\\S2\\TER\\analyse\\src"

pathOutcome = os.path.join(genetic_path,"outcome")
pathGeneticScript =   os.path.join(genetic_path,"genetic-script-data")


In [ ]:
## INIT SETTING

settings_path = os.path.join(pathGeneticScript,"settings-genetic.json")
settings = json.load(open(settings_path))

## CREATE DATAFRAME SETTING

life_entities= [[settings["leekStats"]["life"]],[settings["leekStats"]["life"]]]


## GENERATION  SETTING
nbMaxRow = 250000
nbPartTurn = 32

len_population = settings["population"]
len_maxturn = settings["mapStats"]["max_turns"]
nbGeneration=settings["generations"]

combat_quantity=settings["combatsQuantity"]

## BATCH SOTRING SETTING


len_part_turn = len_maxturn // nbPartTurn
u_tmp = nbMaxRow //(len_part_turn*len_population*combat_quantity)
nbPartGen = u_tmp if u_tmp <nbGeneration and   nbGeneration //u_tmp > 100  else 100
len_part_gen =  nbGeneration//nbPartGen

dict_len_part={"LEN_PART_GEN": len_part_gen, "LEN_PART_TURN":len_part_turn}



## BATCH PARAM 

root_path_gen_turn = init_folder_genSplit(nbPartGen,nbPartTurn,os.path.join(os.getcwd(),"outcome_gen_turn"))

N = 1000

len_batch = 25




In [20]:
## LAUNCH 

init_resume_folder(root_path_gen_turn)


In [ ]:
treat_files_byBatch(life_entities,str_path_outcome=pathOutcome ,dict_len_part=dict_len_part,nbPartTurn=nbPartTurn,root_path_gen_turn=root_path_gen_turn,len_batch = 25)

In [ ]:
batch_1[0]="31_4_12-62_1681316162306.json"
batch_1[1]="31_5_12-62_1681316162306.json"

In [ ]:
print(len_part_gen*len_part_turn*len_population) # <=250000
print(nbPartGen)
print(root_path_gen_turn)
print(len_part_gen)
print(len_part_turn)

In [ ]:
print(count_batch_outcome*len_batch)
print(count_f_outcome)
print (len([name for name in os.listdir(str_path) if os.path.isfile(os.path.join(str_path, name))]))

In [ ]:
#df_cpy["ATM_LIFE_2"] = df_cpy["ATM_LIFE"].rolling(window=2).apply(_funct_2 , raw=True)
#df_cpy["ATM_LIFE"].rolling(1).apply(lambda lst_lst : list( map(add,lst_lst[0],lst_lst[1])),axis=1)
df_cpy.loc[len(df_cpy)-nb_elem*2::1]

In [ ]:
ckd = dict({"MEAN":10.0,"VAR":2.2,"STD_DEV":1.01})
ckd.update(dict_part)
print(ckd)

In [ ]:
'''
def _funct_2 (x_lst):
    _x = x_lst[0]
    for x in x_lst[2::2] :
          _x=list( map(add , _x , x))
    return _x 
'''

In [ ]:
rda.dict_dfValue[str(10)]["name_fields"].index("path")